In [17]:
import cv2
import numpy as np
import fitz
import easyocr
from fpdf import FPDF
import os

In [18]:
reader = easyocr.Reader(['tr','en'], gpu=False)

Using CPU. Note: This module is much faster with a GPU.


In [19]:
def process_single_image(image_cv):
    hsv = cv2.cvtColor(image_cv, cv2.COLOR_BGR2HSV)

    lower_yellow = np.array([20, 100, 100])
    upper_yellow = np.array([30, 255, 255])

    lower_blue = np.array([85, 50, 50])
    upper_blue = np.array([140, 255, 255])

    lower_pink = np.array([140, 50, 50])
    upper_pink = np.array([170, 255, 255])

    lower_green = np.array([35, 50, 50])
    upper_green = np.array([85, 255, 255])

    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)
    mask_blue = cv2.inRange(hsv, lower_blue, upper_blue)
    mask_pink = cv2.inRange(hsv, lower_pink, upper_pink)
    mask_green = cv2.inRange(hsv, lower_green, upper_green)

    mask = cv2.bitwise_or(mask_yellow, mask_blue)
    mask = cv2.bitwise_or(mask, mask_pink)
    mask = cv2.bitwise_or(mask, mask_green)

    kernel_horizontal = np.ones((1, 15), np.uint8)
    kernel_vertical = np.ones((3, 1), np.uint8)

    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel_horizontal)
    mask = cv2.dilate(mask, kernel_vertical, iterations=1)

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=lambda c: cv2.boundingRect(c)[1])

    page_texts = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)

        if w > 40 and h > 15:
            roi = image_cv[y:y+h, x:x+w]
            result = reader.readtext(roi, detail=0, paragraph=True, contrast_ths=0.2)

            if result:
                page_texts.append(" ".join(result))

    return page_texts


In [20]:
def start_mark_extract(file_path):
    all_extracted_texts = []
    
    if file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
        print(f"🖼️ Görsel işleniyor: {file_path}")
        img = cv2.imread(file_path)
        all_extracted_texts = process_single_image(img)
        
    elif file_path.lower().endswith('.pdf'):
        print(f"📄 PDF işleniyor: {file_path}")
        doc = fitz.open(file_path)
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            pix = page.get_pixmap(matrix=fitz.Matrix(2, 2))
            img = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.h, pix.w, pix.n)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            
            texts = process_single_image(img)
            all_extracted_texts.extend(texts)
            print(f"   Sayfa {page_num + 1} tamamlandı.")
            
    return all_extracted_texts

In [34]:
def save_results_to_pdf(text_list, output_name="MarkExtract_Final_Report.pdf"):
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()

    font_path = "C:/Windows/Fonts/arial.ttf"

    if os.path.exists(font_path):
        pdf.add_font("ArialTR", "", font_path, uni=True)
        pdf.set_font("ArialTR", size=12)
    else:
        pdf.set_font("Arial", size=12)

    full_text = " ".join(text_list)

    pdf.multi_cell(0, 8, full_text)

    pdf.output(os.path.join("..", "output", output_name))
    pdf.output(output_name)
    print(f"✅ Rapor başarıyla oluşturuldu: {output_name}")


In [35]:
input_file = os.path.join("..", "input", "pdf_not.pdf")

sonuclar = start_mark_extract(input_file)

if sonuclar:
    save_results_to_pdf(sonuclar)
else:
    print("❌ Isaretlenmis herhangi bir metin bulunamadi.")

📄 PDF işleniyor: ..\input\pdf_not.pdf
   Sayfa 1 tamamlandı.
   Sayfa 2 tamamlandı.
   Sayfa 3 tamamlandı.
   Sayfa 4 tamamlandı.
   Sayfa 5 tamamlandı.
   Sayfa 6 tamamlandı.
   Sayfa 7 tamamlandı.
   Sayfa 8 tamamlandı.
   Sayfa 9 tamamlandı.
✅ Rapor başarıyla oluşturuldu: MarkExtract_Final_Report.pdf
